# Design



### Challenge Name: fw_protect (/embsec/design/fw_protect)

The purpose of this tool will be to secure the firmware. Eventually,
you will want to keep the firmware confidential and be able to verify its integrity and authenticity. But for now, 
you will implement a firmware protect tool that does not add security. This tool should simply:

    1. Pack the version "0" into a little-endian short
    2. Load the firmware binary from firmware.bin
    3. Pack the size of the firmware into a little-endian short
    4. Append the release message "First version of firmware." to end of firmware (make sure it is null-terminated)
    5. Build and send a binary blob in the following format to the grader:
    
        [ 0x2 ]      [ 0x2 ]      [ variable ]           [ variable ]
        -------------------------------------------------------------------------
        version | firmware size |   firmware   |   release message + null-byte   




In [11]:
from embsec import Serial
import struct

def fw_protect():
    ser = Serial("/embsec/design/fw_protect")
    # Your code goes here!
    version = (struct.pack('<h',0))
    with open('firmware_0.bin','rb') as fw:
        firmware=fw.read()
    firmsize=len(firmware)
    release_mess= "First version of firmware.".encode() + b'\0'
    firm_write =firmware+release_mess
    update = struct.pack("<hh",0,firmsize)
    #struct.pack(f"<hh{firmsize}s{len(release_mess)}s",0,firmsize,firmware,release_mess)
    ser.write(update)
    ser.write(firm_write)
    return ser.read_until()
fw_protect()


b'\x00\x000\x00'


b'embsec{fw_protect_8b817d278c508735}\n'

### Challenge Name: fw_update (/embsec/design/fw_update)

For this lesson you will implement a basic update tool for the design challenge.
The tool will be used to update the device firmware. You should focus on reading in the firmware blob file, 
and constructing a set of frames to send to the bootloader. Pay attention to the structure of these 
frames, as the bootloader is expecting a certain size and format.

The bootloader is expecting frames of the following format:

      [ 0x2 ]    [ variable ]
    ----------------------------
    frame size |   data... 

Process:

1. Read in binary blob
2. Construct frames from blob
3. Send a 'U' for update mode & receive a confirmation 'U'
4. Send frames sequentially to boot loader



In [16]:
from embsec import Serial

def fw_update():
    ser = Serial("/embsec/design/fw_update")
    # Your code goes here!
    with open('firmware_1.blob','rb') as fwb:
        firmwareblob=fwb.read()
    print(firmwareblob[2])
    
    ser.write(b'U')
    ser.read()
    for i in range(0,len(firmwareblob),8):
        frame_data = firmwareblob[i:i+8]
        frame=struct.pack('<h',8)
        ser.write(frame)
        ser.write(frame_data)
    ser.write(struct.pack)
    
    return ser.read_until()
    
fw_update()


40


TypeError: 'builtin_function_or_method' object is not iterable